<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
#!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
!curl -L -o SO_vectors_200.bin "https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   269  100   269    0     0    897      0 --:--:-- --:--:-- --:--:--   902

  0 1386M    0 1108k    0     0   902k      0  0:26:13  0:00:01  0:26:12  902k
  0 1386M    0 1858k    0     0   826k      0  0:28:37  0:00:02  0:28:35  735k
  0 1386M    0 3136k    0     0   965k      0  0:24:30  0:00:03  0:24:27 1004k
  0 1386M    0 4445k    0     0  1045k      0  0:22:37  0:00:04  0:22:33 1104k
  0 1386M    0 5723k    0     0  1093k      0  0:21:38  0:00:05  0:21:33 1151k
  0 1386M    0 7080k    0     0  1136k      0  0:20:49  0:00:06  0:20:43 1194k
  0 1386M    0 7958k    0     0  1094k      0  0:21:37  0:00:07  0:21:30 1213k
  0 1386M    0 8980k    0     0  1090k      0  0:2

In [2]:
from gensim.models.keyedvectors import KeyedVectors
path2vectors = "C:/Users/STEPANJUK/Desktop/Deep Learning - Natural Language Processing/2025/lec_01_Simple_Embeddings/[homework_2025]simple_embeddings/SO_vectors_200.bin"
wv_embeddings = KeyedVectors.load_word2vec_format(path2vectors, binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### ***Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [5]:
# method most_simmilar
# Получим список из 10 самых близких слов к слову "dog".
dog_most_simmilar = wv_embeddings.most_similar("dog")

# Выведим топ-10 самых близких слов и оценку близости для каждого из них.
place_number = 1
for simmilar in dog_most_simmilar:
    print(place_number,": " , simmilar)
    place_number += 1

# Рассчитаем близость для эмбедингов слов "dog" и "cat".
similarity_dog2cat = wv_embeddings.similarity("dog", "cat")
print("'dog' & 'cat' similarity: ", similarity_dog2cat)

1 :  ('animal', 0.8564180135726929)
2 :  ('dogs', 0.7880867123603821)
3 :  ('mammal', 0.7623804211616516)
4 :  ('cats', 0.7621253728866577)
5 :  ('animals', 0.760793924331665)
6 :  ('feline', 0.7392397522926331)
7 :  ('bird', 0.7315488457679749)
8 :  ('animal1', 0.7219215631484985)
9 :  ('doggy', 0.7213349342346191)
10 :  ('labrador', 0.7209131717681885)
'dog' & 'cat' similarity:  0.68523407


***Ваш ответ:*** 'cat' не входит в топ-5 ближайших слов к 'dog'.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [6]:
import numpy as np
"""
    Библиотека re позволяет искать, заменять и сопоставлять строки на основе 
    определенного шаблона для работы с текстовыми данными.
"""
import re

# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer

"""
    Tokenizer в NLP используется для разделения текста на отдельные единицы (токены), такие как слова, фразы или предложения.
    
    Токенизация помогает преобразовать текст в структурированные данные представив текст в виде отдельных токенов, которые можно
    использовать для анализа, классификации, извлечения информации и других задач NLP.
"""
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        """
            Функция findall() возвращает все совпадения c шаблоном pattern в строке string в виде списка 
            строк или список кортежей. Строка сканируется слева направо, и совпадения возвращаются в найденном порядке.
        """
        return re.findall('\w+', text)


tokenizer = MyTokenizer()

In [7]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """

    # Разбиваем строку на отдельные слова(токены).
    tokens = tokenizer.tokenize(question)
    # Печатаем токены.
    #print(tokens)

    # Получим список эмбедингов для тех токенов из строки переданной в функцию, которые содержаться в "словаре предобученных векторных представлений".
    wv_tokens = [embeddings[token] for token in tokens if token in embeddings]

    # Обьявляем нулевой вектор для эмбединга строки, как среднего значения всех эмбедтнгов для токенов в строке.
    tokens_mean_value = np.zeros(dim)

    # Если "словарь эмбедтнгов" не содержит эмбедингов для токенов из строки переданной в функцию, 
    # то функция возвращает нулевой вектор.
    if not wv_tokens:
        
        return tokens_mean_value
    # В противном случае рассчитываем среднее значение всех эмбедтнгов для токенов в строке.
    else:    
        # Складываем все эмбедтнги токенов в строке чтобы получить сумму всех эмбедтнгов.
        for wv_t in wv_tokens:
            tokens_mean_value += wv_t
        # Получаем среднее значение поделив сумму всех эмбедтнгов на их кол-во.
        tokens_mean_value = tokens_mean_value/len(wv_tokens)

        return tokens_mean_value

Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

# Третья компонента вектора предложения со значением 2.хх:
# индекс: 34
# значение:  2.76

In [8]:
# Предложение
question = "I love neural networks"

# Получим эмбедтнг предложения.
embedding_of_sentence = question_to_vec(question, wv_embeddings, tokenizer)

# Выводим компоненты вектора предложения со значением 2.хх. 
element_index = 0
for el in embedding_of_sentence:
    if el >= 2 and el < 3:
        print(element_index, ": ", round(el, 2))
    element_index += 1

14 :  2.21
28 :  2.01
34 :  2.76
43 :  2.49
53 :  2.38
87 :  2.3
99 :  2.65
157 :  2.05


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

Hits@K  измеряет долю вопросов, для которых правильный ответ попал в топ-K позиций среди отранжированных кандидатов.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа.

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1]$

Проверяем условие $ \text{rank}_{q'_1} \leq 1 $: ***условие неверно***.

Следовательно, $[\text{rank}_{q'_1} \leq 1] = 0$.

- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Проверяем условие $ \text{rank}_{q'_1} \leq 4 $: ***условие верно***.

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.
* $$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$




# [K = 10] $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}}$ = 0.30103 ~ 0.3



#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

Теперь вычислим метрику **DCG@K** для того же примера, где $ N = 3 $ (три вопроса), и для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):

- $ \text{rank}_{q'_1} = 2 $,
- $ \text{rank}_{q'_2} = 5 $,
- $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **DCG@K** для $ K = 1, 5 $.

---
**Для $ K = 1 $:**
Подставим значения:
$$
\text{DCG@1} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{DCG@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$
$$
\boxed{\text{DCG@1} = \frac{1}{3}}.
$$

---


**Для $ K = 5 $:**
Подставим значения:
$$
\text{DCG@5} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{DCG@5} = \frac{1}{3} \cdot (0.631 + 0.387 + 1) = \frac{1}{3} \cdot 2.018 \approx 0.673.
$$

$$
\boxed{\text{DCG@5} \approx 0.673}.
$$

#### ***Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?


$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$


# $ \text{rank}_{q'_1} = 1 $

### [K = 47] $\text{Hits@47} = [1 \le 47] $ = 1

### [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+1)}\cdot[1 \le 1] = \frac{1}{\log_2{2}}$ = 1


# $ \text{rank}_{q'_1} = 2 $

### [K = 47] $\text{Hits@47} = [2 \le 47] $ = 1

### [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = \frac{1}{\log_2{3}} \cdot 0 $ = 0


# $ \text{rank}_{q'_1} = 48 $

### [K = 47] $\text{Hits@47} = [48 \le 47] $ = 0

### [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+48)}\cdot[48 \le 1] = \frac{1}{\log_2{49}} \cdot 0 $ = 0

# Ответ: 
Так как метрика DCG при всех значения $ \text{rank}_{q'_1} > 1 $ равна нулю,
а метрика Hits при всех значения $ \text{rank}_{q'_1} < 47 $ равна единице,
то максимальное значение разницы между двумя метриками будет равно единице при $ 1 < \text{rank}_{q'_1} < 47 $.
### При $ \text{rank}_{q'_1} = 2 $ :
### Hits@47 - DCG@1 = 1 - 0 = 1





### -------------------------------------------------------------------------------------
Если вопрос какой максимум баллов для метрик Hits@K при K = 47 и для DCG@K при K = 1, то
Max Hits@47 = 1
Max DCG@1 = $\frac{1}{\log_2{1+1}}$ = 1

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.

$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [9]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть Hits@k
    """
    # Обьявляем массив для хранения значений метрики hits для каждого дубликата в ранжированных списках.
    list_of_hits_values = []

    # Проходим через все ранжированные списки, для соответствующего дупликата в этом списке рассчитываем метрику hits
    # и добавляем её в соответствующий массив.
    for rank in dup_ranks:
        if rank <= k:
            list_of_hits_values.append(1)
        else:
            list_of_hits_values.append(0)

    # Рассчитываем среднее значение метрики hits для всего набора  ранжированных списков.
    hits_value = np.sum(list_of_hits_values) / len(list_of_hits_values)
    
    return hits_value

In [10]:
dup_ranks = [2]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0.0
Hits@4 = 1.0


In [11]:
import math

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть DCG@k
    """
    # Обьявляем массив для хранения значений метрики dcg для каждого дубликата в ранжированных списках.
    list_of_dcg_values = []
    
    # Проходим через все ранжированные списки, для соответствующего дупликата в этом списке рассчитываем метрику dcg
    # и добавляем её в соответствующий массив.
    for rank in dup_ranks:
        if rank <= k:
            list_of_dcg_values.append(1 / (np.log2(1+rank)))
        else:
            list_of_dcg_values.append(0)
            
    # Рассчитываем среднее значение метрики dcg для всего набора ранжированных списков.
    dcg_value = np.sum(list_of_dcg_values) / len(list_of_dcg_values)
    
    return dcg_value

In [12]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@10 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@10 = 0.631


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [13]:
import pandas as pd

In [14]:
# Функция для поиска ранга дублирующего предложения в ранжированном списке.
def find_dup_ranks(right_answers, candidates_ranking):
    
    # Обьявляем массив для хранения ранга дубликатов в ранжированных списках.
    list_of_dup_ranks = []
    # Переменная для итерации по массиву ранжированных списков.
    iter_candidates = 0
    for answer in right_answers:
        # Переменная для поиска ранга правильного ответа в ранжированном списке.
        right_rang = 1
        for candidate in candidates_ranking[iter_candidates]:
            if answer == candidate:
                list_of_dup_ranks.append(right_rang)
            right_rang += 1
        iter_candidates += 1
    
    return list_of_dup_ranks

In [15]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = find_dup_ranks(copy_answers, candidates_ranking)

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [16]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [17]:
#!unzip stackoverflow_similar_questions.zip
data_path = "C:/Users/STEPANJUK/Desktop/Deep Learning - Natural Language Processing/2025/lec_01_Simple_Embeddings/[homework_2025]simple_embeddings/data"

Считайте данные.

In [18]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [19]:
validation_data = read_corpus(data_path + '/validation.tsv')

Кол-во строк

In [20]:
len(validation_data)

3760

Размер нескольких первых строк

In [21]:
for i in range(25):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001
6 1001
7 1001
8 1001
9 1001
10 1001
11 1001
12 1001
13 1001
14 1001
15 1001
16 1001
17 1001
18 1001
19 1001
20 1001
21 1001
22 1001
23 1001
24 1001
25 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [23]:
# Функция norm вычисляет норму (или "длину") вектора или матрицы.
# norm([3, 4]) = √(3² + 4²) = 5
from numpy.linalg import norm

# Коссинусная близость.
def custom_cosine_similarity(v1, v2):
    # Если невозможно рассчитать норму вектора, то вернуть 0.
    if not norm(v2):
        return 0
    if not norm(v1):
        return 0
    # Рассчитать и вернуть коссинусную близость.
    else:
        return v1 @ v2.T / norm(v1) / norm(v2)

In [24]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """

    # Получим векторное представление (эмбэдинг) для строки "question"
    # рассчитав среднее значение для эмбэдингов всех слов в строке.
    embedding_of_question = question_to_vec(question, embeddings, tokenizer)

    # Обьявляем массив для хранения индекса каждой строки в списке "candidates".
    candidates_indexes = []

    
    # Обьявляем массив для хранения значений коссинусной близости между эмбэдингом строки "question" 
    # и эмбэдингами строк в списке "candidates"
    list_of_cos_sim = []
    # Пройдём через список "candidates" чтобы заполнить массив "candidates_indexes" индексами соответствующих строк,
    # а так же рассчитать коссинусную близость между эмбэдингом строки "question" 
    # и эмбэдингами строк в списке "candidates"
    idx = 0
    for candidate in candidates:
        #candidates_indexes.append(idx)
        candidates_indexes.append((idx, candidate))
        list_of_cos_sim.append(custom_cosine_similarity(embedding_of_question, question_to_vec(candidate, embeddings, tokenizer)))
        #list_of_cos_sim.append(np.sum(cosine_similarity(embedding_of_question.reshape(-1, 1), 
        #                                                question_to_vec(candidate, embeddings, tokenizer).reshape(-1, 1))) / 200)
        idx += 1






    # Количество строк в списке "candidates".
    amount_of_candidates = len(list_of_cos_sim)
    # Отсортируем список "candidates_indexes" при помощи сортировки пузырьком чтобы каждый индекс строки стоял
    # на месте определённом при помощи коссинусной близости.
    for i in range(amount_of_candidates - 1):
        for j in range(amount_of_candidates - i - 1):
            if list_of_cos_sim[j] < list_of_cos_sim[j+1]:
                list_of_cos_sim[j], list_of_cos_sim[j+1] = list_of_cos_sim[j+1], list_of_cos_sim[j]
                candidates_indexes[j], candidates_indexes[j+1] = candidates_indexes[j+1], candidates_indexes[j]  

    return candidates_indexes

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [25]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

list_of_ranks = rank_candidates(questions[0], candidates[0], wv_embeddings, tokenizer)
print(list_of_ranks)

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]


In [26]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [27]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

SyntaxError: invalid syntax (1724534416.py, line 5)

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

# Ответ: для эксперимента 2 ответ так же будет 102


Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [28]:
from tqdm.notebook import tqdm

In [29]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [30]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.879
DCG@1000: 0.444 | Hits@1000: 1.000


Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [31]:
train_data = read_corpus(data_path + '/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [32]:
train_data[111258]

['Determine if the device is a smartphone or tablet?',
 'Change imageView params in all cards together']

In [33]:
# Создаем общий корпус текстов
corpus = [tokenizer.tokenize(text[0]) for text in train_data]

'''your code'''

'your code'

In [34]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window= 3,                # Размер окна контекста
    min_count= 3,             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [35]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [36]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.198 | Hits@   1: 0.198
DCG@   5: 0.252 | Hits@   5: 0.302
DCG@  10: 0.271 | Hits@  10: 0.363
DCG@ 100: 0.321 | Hits@ 100: 0.609
DCG@ 500: 0.354 | Hits@ 500: 0.871
DCG@1000: 0.368 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Tokenizer в NLP используется для разделения текста на отдельные единицы (токены), такие как слова, фразы или предложения. В тетради было предложено использовать библиотеку re, которая позволяет искать, заменять и сопоставлять строки на основе определенного шаблона для работы с текстовыми данными. Функция findall() возвращает все совпадения c шаблоном pattern в строке string в виде списка строк или список кортежей. Строка сканируется слева направо, и совпадения возвращаются в найденном порядке.

Нормализация слов помогает сократить размер словаря благодаря таким методам как лемматизация, стемминг, приведение к нижнему регистру, удаление пунктуации и прочим. Что в свою очередь помогает сделать вычесления проще.

Предобученные векторные представления слов на постах Stack Overflow показали лучший результат, чем обученные мной на предоставленных вами данных.

Думаю, что большее кол-во эпох, а так же большее кол-во тренировочных данных должны поправить ситуацию в лучшую сторону.

Я бы попробовал построить энкодер для получения эмбедингов слов.